### Langchain Tutorial: Chatbot with Persistent Memory

https://python.langchain.com/docs/tutorials/chatbot/

In [1]:
!pip install -qU langchain-core langgraph>0.2.27

In [2]:
# API keys and environment variables
import getpass
import os

# LangSmith can be used to debug / test / monitor AI Application 

os.environ["LANGSMITH_TRACING"] = "true"
if not os.environ.get("LANGSMITH_API_KEY"):
  os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API key for LangSmith: ")

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

In [3]:
# Load model
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [4]:
# Add in-memory checkpoint (Wrap with simple langraph application)

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [5]:
# Add configuration

# A thread is a unique ID or thread identifier assigned to each checkpoint saved by a checkpointer. 
# It contains the accumulated state of a sequence of runs. When a run is executed, the state of the 
# underlying graph of the assistant will be persisted to the thread

# The state of a thread at a particular point in time is called a checkpoint. 

config = {"configurable": {"thread_id": "abc123"}}

In [6]:
# Invoke the application
from langchain_core.messages import HumanMessage

query = "Hi! My Name is Pubudu"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Pubudu! Nice to meet you.

How can I help you today?


In [7]:
# Check if the chatbot has memory
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Pubudu!


In [8]:
# Change the 'thread_id' to start the conversation in fresh
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

As an AI, I don't have access to your personal information, so I don't know your name. You'd have to tell me!


In [9]:
# Revert back to previous thread and it remembers the history
config = {"configurable": {"thread_id": "abc123"}}
query1 = "Write a sentence including my name"

input_messages = [HumanMessage(query1)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Here's a sentence including your name:

**Pubudu** is a unique and interesting name.


In [10]:
# View entire model output
print(output)

{'messages': [HumanMessage(content='Hi! My Name is Pubudu', additional_kwargs={}, response_metadata={}, id='b4a03a2a-2268-41cb-bc71-9e7e6f0d0aad'), AIMessage(content='Hi Pubudu! Nice to meet you.\n\nHow can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--2895f21e-b528-437e-af44-01c045dfa9f2-0', usage_metadata={'input_tokens': 8, 'output_tokens': 257, 'total_tokens': 265, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 240}}), HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='cd4947a1-fa4c-40d5-8620-95061ea8ae28'), AIMessage(content='Your name is Pubudu!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--1225492e